In [1]:
from marked_words import marked_words
import pandas as pd
from collections import Counter
import numpy as np
from konlpy.tag import Mecab
import re
import sklearn.feature_selection
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
def pprint(dic):
    full_list = []
    for word in sorted(dic,key=lambda x: x[1],reverse=True):
        # print("%s, %.2f" % (word[0],word[1]))
        full_list.append(word[0])
    return full_list

def anonymize(bio, replacement=""):
    """
    지역 및 성별을 직접적으로 나타내는 단어를 masking하는 함수
    """
    bio = re.sub(r"남|여|녀|남자|여자|남성|여성|남편|부인|그|그녀", replacement, bio)
    bio = re.sub(r"서울|전라도|경상도|제주도|전라|경상|제주", replacement, bio)
    return bio

In [3]:
df = pd.read_csv('data/gpt3_generations.csv')

gpt3_mw = {}

# 서울을 majority로 보았을 때, 다른 지역의 marked words
for province in df['province'].unique():
    # print('\n Top words for %s \n-------' % province)
    outs = pprint(marked_words(df, [province], ['province'], ['서울']))
    gpt3_mw[province] = outs

# 다른 지역을 각각 majority로 보았을 때, 서울의 marked words
temps = []
for province in df["province"].unique():
    # print('\n Top words for %s \n-------' % province)
    temp = pprint(marked_words(df, ["서울"], ["province"], [province]))
    temps.extend(temp)
seen = Counter(temps).most_common()
gpt3_mw["서울"] = [w for w, c in seen if c == 3]

In [4]:
# 남자를 majority로 보았을 때, 여자의 marked words
for gender in df["gender"].unique():
    # print('\n Top words for %s \n-------' % gender)
    outs = pprint(marked_words(df, [gender], ["gender"], ["남자"]))
    gpt3_mw[gender] = outs

# 여자를 majority로 보았을 때, 남자의 marked words
temps = []
for gender in df["gender"].unique():
    # print('\n Top words for %s \n-------' % gender)
    temp = pprint(marked_words(df, ["남자"], ["gender"], [gender]))
    temps.extend(temp)
seen = Counter(temps).most_common()
gpt3_mw["남자"] = [w for w, c in seen if c == 1]


In [5]:
# 서울 남자를 majority로 보았을 때, 다른 intersectional 집단의 marked words
for province in df["province"].unique():
    for gen in df["gender"].unique():
        gpt3_mw[f"{province} {gen}"] = pprint(
            marked_words(df, [province, gen], ["province", "gender"], ["서울", "남자"])
        )

# 다른 intersectional 집단을 majority로 보았을 때, 서울 남자의 marked words
temps = []
for province in df["province"].unique():
    for gender in df["gender"].unique():
        # print('\n Top words for %s \n-------' % gender)
        temp = pprint(marked_words(df, ["서울", "남자"], ["province", "gender"], [province, gender]))
        temps.extend(temp)
seen = Counter(temps).most_common()
gpt3_mw["서울 남자"] = [w for w, c in seen if c == 4 * 2 - 1]

In [6]:
for key in gpt3_mw.keys():
    print(f"{key}의 marked word 개수 : {len(gpt3_mw[key])}")


서울의 marked word 개수 : 26
제주도의 marked word 개수 : 35
경상도의 marked word 개수 : 27
전라도의 marked word 개수 : 18
여자의 marked word 개수 : 26
남자의 marked word 개수 : 24
서울 여자의 marked word 개수 : 0
서울 남자의 marked word 개수 : 0
제주도 여자의 marked word 개수 : 18
제주도 남자의 marked word 개수 : 6
경상도 여자의 marked word 개수 : 8
경상도 남자의 marked word 개수 : 1
전라도 여자의 marked word 개수 : 5
전라도 남자의 marked word 개수 : 2


In [7]:
# SVM으로 직접적으로 집단에 대한 정보를 제공하는 단어를 masking한 뒤, 각 집단을 분류하는 성능을 측정

mecab = Mecab()

vectorizer = CountVectorizer(binary=True, decode_error="ignore")
tokenizer = vectorizer.build_tokenizer()


def anonymize(bio, replacement=""):
    """
    지역 및 성별을 직접적으로 나타내는 단어를 masking하는 함수
    """
    bio = re.sub(r"남|여|녀|남자|여자|남편|부인|아내|그|그녀", replacement, bio)
    bio = re.sub(r"서울|전라도|경상도|제주도|전라|경상|제주", replacement, bio)
    return bio


df_copy = df.copy()
df_copy["provincegender"] = df_copy["province"] + df_copy["gender"]
data = (
    df_copy["text"]
    .apply(lambda s: " ".join(mecab.morphs(s)))
    .str.lower()
    .replace("[^\w\s]", "", regex=True)
)

top_words = dict()
dv3_svm = {}
for st in ["province", "gender", "provincegender"]:
    print(st.upper())
    concept_data = [anonymize(d) for d in data]
    labels = df_copy[st]
    bios_data_train, bios_data_test, Y_train, Y_test = train_test_split(
        concept_data, labels, test_size=0.2, random_state=42, stratify=labels
    )
    vectorizer = CountVectorizer(analyzer="word", min_df=0.001, binary=False)
    X_train = vectorizer.fit_transform(bios_data_train)
    X_test = vectorizer.transform(bios_data_test)
    accs = []
    feature_names = vectorizer.get_feature_names_out()
    for r in df_copy[st].unique():
        svm = SVC(kernel="linear")
        Y_train_bin = Y_train == r
        svm.fit(X_train, Y_train_bin)
        acc = sklearn.metrics.accuracy_score(Y_test == r, svm.predict(X_test))
        # print("%s Accuracy: %.2f"%(r, acc))
        accs.append(acc)
        coef = svm.coef_.toarray()[0]
        _, names = zip(*sorted(zip(coef, feature_names)))
        # print("Top 10 words: %s" % str(names[-10:][::-1]))
        dv3_svm[r] = names[-10:][::-1]
    print(
        "Mean accuracy across %s groups: %.2f ± %.2f"
        % (st, np.mean(accs), np.std(accs))
    )

PROVINCE
Mean accuracy across province groups: 0.90 ± 0.05
GENDER
Mean accuracy across gender groups: 0.70 ± 0.00
PROVINCEGENDER
Mean accuracy across provincegender groups: 0.90 ± 0.02
